## Start

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from imports import *
from keras.applications import xception

Using TensorFlow backend.


In [3]:
bs=64; sz=149; lr=2e-3
path = "/data/jhoward/fast/dogscats/"

In [4]:
gen = image.ImageDataGenerator(preprocessing_function=xception.preprocess_input)

## 1. Fine-tune last layer of full network

In [25]:
mn=Xception(include_top=False, input_shape=(sz,sz,3), pooling='avg')
for l in mn.layers[:-1]: l.trainable=False
for l in mn.layers: 
    if isinstance(l, BatchNormalization):
        l._updates=[]
        l._per_input_updates={}

In [17]:
outp = Dense(2, activation='softmax')(mn.output)
m = Model(mn.input, outp)
K.set_learning_phase(0)
m.compile(SGD(lr, momentum=0.9), 'categorical_crossentropy', metrics=['accuracy'])

In [19]:
trn_batches = gen.flow_from_directory(f'{path}train', (sz,sz), batch_size=bs)
val_batches = gen.flow_from_directory(f'{path}valid', (sz,sz), batch_size=bs, shuffle=False)
nb_trn = math.ceil(trn_batches.n/bs)
nb_val = math.ceil(val_batches.n/bs)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [20]:
m.fit_generator(trn_batches, nb_trn, workers=1, epochs=2,
                 validation_data=val_batches, validation_steps=nb_val)

Epoch 1/2
360/360 [==============================] - 58s - loss: 0.1078 - acc: 0.9550 - val_loss: 0.0811 - val_acc: 0.9635
Epoch 2/2
360/360 [==============================] - 56s - loss: 0.0759 - acc: 0.9704 - val_loss: 0.0752 - val_acc: 0.9675


In [39]:
for l in m.layers: l.trainable=True
for l in m.layers[:50]: l.trainable=False

In [40]:
m._collected_trainable_weights = m.trainable_weights
m.train_function = None

In [41]:
m.compile(SGD(lr/10, momentum=0.9), 'categorical_crossentropy', metrics=['accuracy'])

In [42]:
m.fit_generator(trn_batches, nb_trn, workers=1, epochs=2,
                 validation_data=val_batches, validation_steps=nb_val)

Epoch 1/2
360/360 [==============================] - 100s - loss: 0.0595 - acc: 0.9787 - val_loss: 0.0659 - val_acc: 0.9695
Epoch 2/2
360/360 [==============================] - 99s - loss: 0.0510 - acc: 0.9819 - val_loss: 0.0625 - val_acc: 0.9720


## 2. Pre-compute output of penultimate layer and train single layer net

In [10]:
mn=Xception(include_top=False, input_shape=(sz,sz,3), pooling='avg')

Precompute pooling output:

In [11]:
fix_batches = gen.flow_from_directory(f'{path}train', (sz,sz), batch_size=bs, shuffle=False)
val_batches = gen.flow_from_directory(f'{path}valid', (sz,sz), batch_size=bs, shuffle=False)

trn_acts = mn.predict_generator(generator=fix_batches, verbose=1, 
                                steps=nb_trn, workers=1)
val_acts = mn.predict_generator(generator=val_batches, verbose=1,
                                steps=nb_val, workers=1)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
31/32 [============================>.] - ETA: 0s

Train single layer:

In [12]:
inp = Input(batch_shape=mn.output_shape)
outp = Dense(1, activation='sigmoid')(inp)
fc = Model(inp, outp)
fc.compile(SGD(lr, momentum=0.9), 'binary_crossentropy', metrics=['accuracy'])

In [13]:
fc.fit(trn_acts, fix_batches.classes, bs, 2, validation_data=(val_acts, val_batches.classes))

Train on 23000 samples, validate on 2000 samples
Epoch 1/2
23000/23000 [==============================] - 1s - loss: 0.1184 - acc: 0.9540 - val_loss: 0.0836 - val_acc: 0.9690
Epoch 2/2
23000/23000 [==============================] - 0s - loss: 0.0802 - acc: 0.9693 - val_loss: 0.0774 - val_acc: 0.9710
